<a href="https://colab.research.google.com/github/Muzhi1920/awesome-models/blob/main/05-%E7%89%B9%E5%BE%81%E4%BA%A4%E4%BA%92/08_DeepCrossing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D, DenseFeatures, ReLU

## 0.准备工作

In [ ]:
seq = fc.categorical_column_with_hash_bucket('seq', hash_bucket_size=10, dtype=tf.int64)
target = fc.categorical_column_with_hash_bucket('target', hash_bucket_size=10, dtype=tf.int64)
seq_col = fc.embedding_column(seq, dimension=8)
target_col = fc.embedding_column(target, dimension=8)
columns = [seq_col, target_col]
features={
  "seq": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
      values=[1100, 1101, 1102, 1101, 1103],
      dense_shape=[3, 2]),
  "target": tf.sparse.SparseTensor(
      indices=[[0, 0],[1,0],[2,0]],
      values=[1102,1103,1100],
      dense_shape=[3, 1]),

}
tf.sparse.to_dense(features['seq'])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1100, 1101],
       [1102, 1101],
       [1103,    0]], dtype=int32)>

In [ ]:
input_layer = DenseFeatures(columns, name='input_layer')
net = input_layer(features)
net

<tf.Tensor: shape=(3, 16), dtype=float32, numpy=
array([[ 1.7270304e-01, -2.6593623e-01, -3.8202830e-02,  6.3266888e-02,
        -8.7955520e-02, -1.2240441e-01,  8.0541268e-02, -7.6435603e-02,
         4.3260914e-01, -4.4453360e-02, -4.1503379e-01, -1.3240685e-01,
         1.9923095e-01, -5.9006828e-01,  7.6168798e-02, -3.7161762e-01],
       [ 1.7802976e-04, -5.1683202e-02,  1.2500286e-01,  2.9188102e-01,
         1.2777559e-03, -5.4559555e-02,  1.8974549e-01,  2.4603914e-01,
        -1.6755906e-01,  4.7612214e-01, -9.8921970e-02,  1.1331042e-01,
        -1.4801481e-01,  6.6043340e-02, -1.7649618e-01, -4.1666273e-02],
       [-3.4509450e-01, -2.0577022e-01, -2.8365418e-01, -3.4777902e-02,
         5.2838224e-01,  1.1012834e-01, -4.2458156e-01,  6.6934839e-02,
         1.4291008e-01,  4.1550305e-01, -2.9657465e-01, -6.3238835e-01,
         1.8077403e-01,  1.5333142e-02, -1.7123570e-01,  2.6142594e-01]],
      dtype=float32)>

In [ ]:
class res_layer(Layer):
    def __init__(self, units):
        """
        :param hidden_unit: A list. Neural network hidden units.
        :param dim_stack: A scalar. The dimension of inputs unit.
        """
        super(res_layer, self).__init__()
        self.layer1 = Dense(units=units[0], activation='relu')
        self.layer2 = Dense(units=units[1], activation=None)
        self.relu = ReLU()

    def call(self, inputs, **kwargs):
        x = inputs
        x = self.layer1(x)
        x = self.layer2(x)
        outputs = self.relu(x + inputs)
        return outputs

![](https://camo.githubusercontent.com/f60a4c8984353ba7f2f73e0cff66f4a98c77aed58b8afeae69fdacb21d68ea98/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f426c61636b5370616365475a592f63646e2f696d672f74665f362e706e67)

In [ ]:
hidden_units = [16, tf.shape(net)[1]]
nums_stack = 2
res_layers = [res_layer(hidden_units) for num in range(nums_stack)]
output_layer = Dense(1, activation='sigmoid')

In [ ]:
for res in res_layers:
  net = res(net)
outputs = output_layer(net)

In [ ]:
outputs

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.49188623],
       [0.52558416],
       [0.49548498]], dtype=float32)>